In [ ]:
import subprocess
import re
import pandas as pd
import shlex
import os
from datetime import datetime
import io
import sys

In [ ]:
# Reference:
# https://askubuntu.com/questions/487206/dconf-change-a-string-key

# Example Commands:

# dconf read /org/gnome/settings-daemon/plugins/media-keys/custom-keybindings/custom0/binding
# dconf read /org/gnome/settings-daemon/plugins/media-keys/custom-keybindings/custom0/command
# dconf read /org/gnome/settings-daemon/plugins/media-keys/custom-keybindings/custom0/name

# dconf write /org/gnome/settings-daemon/plugins/media-keys/custom-keybindings/custom0/name "'Terminal'"

# gsettings set org.gnome.desktop.wm.keybindings close  "['<Alt>F4', '<Shift><Super>q']"

# gsettings get org.gnome.settings-daemon.plugins.media-keys custom-keybindings

In [ ]:
def dconf_list():
    dconf_cmd = 'dconf list /org/gnome/settings-daemon/plugins/media-keys/custom-keybindings/'
    dconf_res = subprocess.check_output(shlex.split(dconf_cmd),universal_newlines=True)
    dconf_lst = re.findall('custom\d+', dconf_res)
    dconf_lst.sort()
    return dconf_lst

In [ ]:
def dconf_confirm(key):
    dconf_cmd = f'dconf list /org/gnome/settings-daemon/plugins/media-keys/custom-keybindings/{key}/'
    dconf_res = subprocess.check_output(shlex.split(dconf_cmd),universal_newlines=True)
    dconf_lst = dconf_res.strip().split('\n')
    assert dconf_lst == ['binding', 'command', 'name'], f'unexpected fields from result of: {dconf_cmd}'

In [ ]:
def dconf_read(key,field):
    dconf_cmd = f'dconf read /org/gnome/settings-daemon/plugins/media-keys/custom-keybindings/{key}/{field}'
    dconf_res = subprocess.check_output(shlex.split(dconf_cmd),universal_newlines=True)
    dconf_res = dconf_res.strip()
    #print(dconf_res)
    assert (dconf_res[0] == "'") and (dconf_res[-1] == "'"), f'{dconf_cmd}-> {dconf_res}'
    return dconf_res[1:-1]

In [ ]:
def dconf_entry(key):
    dconf_confirm(key)
    home = os.path.expandvars('$HOME')
    key_num = int(key.replace('custom',''))
    name = dconf_read(key,'name')
    # replace $HOME with ~ while processing
    command = dconf_read(key,'command').replace(home,'~')
    binding = dconf_read(key,'binding')
    return key_num, key, name, command, binding

In [ ]:
def dconf_to_df():
    data = []
    names = dconf_list()
    for name in names:
        data.append(list(dconf_entry(name)))
    FIELDS = ['key_num', 'key', 'name','command','binding']
    df = pd.DataFrame(data,columns=FIELDS)
    df = df.set_index('key_num').sort_index()
    return df   

In [ ]:
def df_append_entry(df,name,command,binding):    
    # see if gap in key_num (i.e., customX)
    key_num_set = set(df.index)
    range_set = set(range(len(df.index)))
    diff_set = key_num_set - range_set
    
    # if no gap, set to next number, otherwise use first gap
    if diff_set == set([]):    
        key_num = len(df)
    else:
        key_num = list(diff_set)[0]
    key = f'custom{key_num}'
    
    # append key
    df.loc[key_num] = [key,name,command,binding]
    df = df.sort_index()
    return df

In [ ]:
def df_append_entry_if_unique(df,name,command,binding):
    if name not in list(df['name']):
        df = df_append_entry(df,name,command,binding)
        print(f'adding new entry for: {name}')
    else:
        print(f'skipping entry, detected entry with same name: {name}')
    return df

In [ ]:
def dconf_reset():
    dconf_cmd = 'dconf reset -f /org/gnome/settings-daemon/plugins/media-keys/custom-keybindings/'
    dconf_res = subprocess.check_output(shlex.split(dconf_cmd),universal_newlines=True)

In [ ]:
def dconf_write(key,field,value):
    dconf_cmd = f'dconf write /org/gnome/settings-daemon/plugins/media-keys/custom-keybindings/{key}/{field} "\'{value}\'"'
    dconf_res = subprocess.check_output(shlex.split(dconf_cmd),universal_newlines=True)

In [ ]:
def df_to_dconf(df):
    dconf_reset()
    
    home = os.path.expandvars('$HOME')
    
    for row in df.iloc:
        key,name,command,binding = tuple(row)
        # expand ~, since dconf needs absolute paths
        command = command.replace('~',home)
        
        dconf_write(key,'name',name)
        dconf_write(key,'command',command)
        dconf_write(key,'binding',binding)    

In [ ]:
def gsettings_get():
    gsettings_cmd = 'gsettings get org.gnome.settings-daemon.plugins.media-keys custom-keybindings'
    gsettings_res = subprocess.check_output(shlex.split(gsettings_cmd),universal_newlines=True)
    return gsettings_res

In [ ]:
def gsettings_list():
    gsettings_res = gsettings_get()
    gsettings_lst = re.findall('custom\d+', gsettings_res)
    #gsettings_lst.sort()
    return gsettings_lst

In [ ]:
def gsettings_set(df):
    value = '[' + ', '.join([f"'/org/gnome/settings-daemon/plugins/media-keys/custom-keybindings/{key}/'" for key in list(df['key'])]) + ']'
    gsettings_cmd = f'gsettings set org.gnome.settings-daemon.plugins.media-keys custom-keybindings "{value}"'
    #print(gsettings_cmd)
    gsettings_res = subprocess.check_output(shlex.split(gsettings_cmd),universal_newlines=True)

In [ ]:
# hotkey_csv
JUPYTER_ARGS = 'hotkeys.csv'

In [ ]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('hotkey_csv', type=str,help='csv config script to apply')
parser.add_argument('--overwrite',default=False,action='store_true',help='force overwrites current GNOME hotkeys')

IN_NOTEBOOK = 'get_ipython' in globals()
IN_TERMINAL = not IN_NOTEBOOK

if IN_NOTEBOOK:
    args = parser.parse_args(JUPYTER_ARGS.split(' ')) # call from notebook
else:
    args = parser.parse_args() # call from command line
    JUPYTER_ARGS = ' '.join(sys.argv[1:])

In [ ]:
HOTKEY_CSV = args.hotkey_csv
OVERWRITE = args.overwrite

In [ ]:
df = dconf_to_df()
df

In [ ]:
assert set(df['key']) == set(gsettings_list()), 'dconf and gsettings do not match before editing'

In [ ]:
# backup current config
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
df.to_csv(f'backup/{timestamp}_hotkeys.csv',index=False)

In [ ]:
with open(HOTKEY_CSV) as READ:
    str_csv = READ.read()

# remove comments
comments = re.findall('#.*\n', str_csv)
for comment in comments:
    str_csv = str_csv.replace(comment,'\n')

# remove multi-lines
multilines = sorted(re.findall('\n\n+',str_csv),key=len,reverse=True)
for multiline in multilines:
    str_csv = str_csv.replace(multiline,'\n')

# remove trailing space
spaces = re.findall(' +\n',str_csv)
for space in spaces:
    str_csv = str_csv.replace(space,'\n')

In [ ]:
# load hotkey config
df_ = pd.read_csv(io.StringIO(str_csv))

# expands ~ and variables (i.e., $DOTFILES_ROOT) in config
df_['command'] = df_['command'].map(lambda x: os.path.expanduser(x)).map(lambda x: os.path.expandvars(x))
df_

In [ ]:
# if OVERWRITE, clear df
if OVERWRITE:
    df = df.drop(df.index)

# append unique entries
for row in df_.iloc:
    name,command,binding = tuple(row)
    df = df_append_entry_if_unique(df,name,command,binding)
df

In [ ]:
# write and check
df_to_dconf(df)
gsettings_set(df)
assert set(df['key']) == set(gsettings_list()), 'dconf and gsettings do not match after editing'

In [ ]:
print('success')